In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import scvelo as scv
import cellrank as cr
import anndata as ad
from cellrank.kernels import ConnectivityKernel, VelocityKernel
from cellrank.estimators import GPCCA
from tueplots import bundles
from tueplots import axes

In [2]:
cr.logging.print_versions()

cellrank==1.5.1+g16069e25 scanpy==1.9.1 anndata==0.8.0 numpy==1.23.5 numba==0.56.4 scipy==1.10.0 pandas==1.5.2 pygpcca==1.0.4 scikit-learn==1.2.0 statsmodels==0.13.5 scvelo==0.2.5 pygam==0.8.0 matplotlib==3.6.2 seaborn==0.12.2


In [3]:
org_adata = sc.read("/lustre/groups/ml01/workspace/monge_velo/data/benchmarks/larry_invitro/larry_invitro_adata_sub_raw.h5ad")
org_adata

AnnData object with n_obs × n_vars = 49302 × 23420
    obs: 'Library', 'Cell barcode', 'time_info', 'Starting population', 'state_info', 'Well', 'SPRING-x', 'SPRING-y'
    var: 'Accession', 'Chromosome', 'End', 'Start', 'Strand'
    uns: 'data_des'
    obsm: 'X_clone', 'X_emb'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [4]:
leave_out = [2]
adata = org_adata[~org_adata.obs['time_info'].isin(leave_out)].copy()

adata.obs = adata.obs.loc[:, ['time_info', 'state_info']]
adata.var = adata.var[[]]
adata.uns = {}

sc.pp.log1p(adata)
scv.pp.filter_and_normalize(adata, min_shared_counts = 20, n_top_genes=2000)

adata

Filtered out 16268 genes that are detected 20 counts (shared).
Normalized count data: spliced, unspliced.
Extracted 2000 highly variable genes.


AnnData object with n_obs × n_vars = 44664 × 2000
    obs: 'time_info', 'state_info', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    var: 'means', 'dispersions', 'dispersions_norm', 'highly_variable'
    uns: 'log1p'
    obsm: 'X_clone', 'X_emb'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced'

In [5]:
adata_four = adata[adata.obs['time_info'] == 4.0, :].copy()
adata_six = adata[adata.obs['time_info'] == 6.0, :].copy()

In [6]:
sc.pp.pca(adata_four, n_comps=50)
scv.pp.moments(adata_four)
scv.tl.recover_dynamics(adata_four, n_jobs=80)
scv.tl.velocity(adata_four, mode = 'dynamical')

computing neighbors
    finished (0:00:45) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:02) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
recovering dynamics (using 80/96 cores)
or disable the progress bar using `show_progress_bar=False`.


/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserW

    finished (0:40:24) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:32) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [7]:
sc.pp.pca(adata_six, n_comps=50)
scv.pp.moments(adata_six)
scv.tl.recover_dynamics(adata_six, n_jobs=80)
scv.tl.velocity(adata_six, mode = 'dynamical')

computing neighbors
    finished (0:00:09) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:05) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)
recovering dynamics (using 80/96 cores)


/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/home/icb/jonas.flor/miniconda3/envs/neuralBenchmark/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:700: UserW

    finished (1:32:43) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)
computing velocities
    finished (0:00:52) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [8]:
adata = ad.concat([adata_four, adata_six])
sc.pp.neighbors(adata)
adata

AnnData object with n_obs × n_vars = 44664 × 2000
    obs: 'time_info', 'state_info', 'initial_size_unspliced', 'initial_size_spliced', 'initial_size', 'n_counts'
    uns: 'neighbors'
    obsm: 'X_clone', 'X_emb', 'X_pca'
    layers: 'ambiguous', 'matrix', 'spliced', 'unspliced', 'Ms', 'Mu', 'fit_t', 'fit_tau', 'fit_tau_', 'velocity', 'velocity_u'
    obsp: 'distances', 'connectivities'

In [9]:
vk = VelocityKernel(adata).compute_transition_matrix()

100%|██████████████████████████████████████████████████████████████████████████| 44664/44664 [01:29<00:00, 497.75cell/s]


In [ ]:
g = GPCCA(vk)
g.compute_schur(n_components=20)
g.plot_spectrum(real_only=True)

In [ ]:
g.compute_macrostates(n_states=8, cluster_key="celltype")
scv.set_figure_params('scvel o', transparent=True, fontsize=20, color_map='viridis')
g.plot_macrostates(discrete=True, basis="umap", legend_loc="right", legend_fontweight='normal', legend_fontsize='12', dpi=250)

In [ ]:
g.set_terminal_states_from_macrostates(
    [
        "Acinar", "Ductal_1", "Ductal_2", "Alpha", "Beta", "Delta", "Epsilon", 
    ]
)
g.compute_absorption_probabilities(solver="gmres", use_petsc=True, tol=1e-12, preconditioner='ilu', time_to_absorption='all')
g.plot_absorption_probabilities(same_plot=False, basis="umap", perc=[0, 99], ncols=2)

In [ ]:
scv.set_figure_params('scvelo', transparent=True, fontsize=20, color_map='viridis')
cr.pl.aggregate_absorption_probabilities(
    adata,
    mode='heatmap',
    lineages=["Acinar", "Ductal_1", "Ductal_2", "Alpha", "Beta", "Delta", "Epsilon", ],
    cluster_key='celltype',
    clusters=['Acinar', 'Multipotent', 'Tip', 'Ductal', 'Ngn3 High early', 'Ngn3 High late', 'Fev+ Alpha', 'Alpha', 'Fev+ Beta', 'Beta', 'Fev+ Delta', 'Delta', 'Fev+ Epsilon', 'Epsilon'],
    figsize=(17, 6),
    title="",
    save="transprobs_pancreas1415_scvelo.png"
)

In [ ]:
adata.uns["velocity_graph"] = vk.transition_matrix

scv.set_figure_params('scvelo', transparent=True, fontsize=10, dpi_save=400,color_map='viridis')
fig = plt.figure()
ax = scv.pl.velocity_embedding_stream(adata, basis="umap", smooth=0.5, title="", legend_loc="none", show=False)
legend =ax.legend(bbox_to_anchor=[1.25, 1.5], loc='upper center', ncol=6,frameon=True, prop={'size': 18})

In [ ]:
scv.tl.velocity_confidence(adata)

In [ ]:
scv.set_figure_params('scvelo', fontsize=20)
scv.settings.presenter_view = False
scv.pl.scatter(adata, color='velocity_confidence',
               perc=[2, 98],
               cmap='gnuplot',
               vmid=0.75,
               rescale_color=[0.5, 1.0],
               size=15,
               title="",
               #save="velconf_pancreas1415_scvelo.png"
              )